In [ ]:
import pandas as pd
from pathlib import Path

## Input dataset

In [ ]:
df = pd.read_excel("../data/external/103_table_v5.xlsx")
df_genomes_meta = pd.read_csv("../data/external/df_genomes_meta.csv")

## Statistics

In [ ]:
print("Total genera:", len(df_genomes_meta.genus_gtdb.value_counts()) -1)
df_genomes_meta.genus_gtdb.value_counts()

There are two unassigned genus:

In [ ]:
df_genomes_meta[df_genomes_meta.genus_gtdb == "g__"]

Based on GTDBtk, There are a total of 26 genera + 2 unassigned genus. Making it in total ~28 genera in the dataset.

In [ ]:
df_g1032 = df_genomes_meta[df_genomes_meta.genus_gtdb != "g__"]
species_gtdb_identified = df_g1032[df_g1032.species_gtdb != "s__"].species_gtdb.value_counts()
#species_gtdb_identified

In [ ]:
species_mash_identified = df_g1032[~df_g1032.Mash_species.isnull()].Mash_species.value_counts()
#species_mash_identified

In [ ]:
f"Total species that can be identified: {len(species_gtdb_identified) + len(species_mash_identified)}. Identifided with GTDBtk: {len(species_gtdb_identified)}. Identifided with MASH: {len(species_mash_identified)}. There are {len(df_genomes_meta[df_genomes_meta.genus_gtdb == 'g__'])} genomes that cannot be assigned."

## Cleanup

In [ ]:
species_assignment = df_genomes_meta.loc[:, ["genome_id", "genus_gtdb", "species_gtdb", "Mash_species"]]
species_assignment['genus_gtdb'] = species_assignment['genus_gtdb'].str.replace('g__', '')
species_assignment['species_gtdb'] = species_assignment['species_gtdb'].str.replace('s__', 'sp.')
species_assignment["assigned_species"] = "(" + species_assignment['Mash_species'].fillna("") + ")"
species_assignment["assigned_species"] = species_assignment['assigned_species'].str.replace("()", "")
species_assignment['species_gtdb'] = species_assignment['genus_gtdb'] + ' ' + species_assignment['species_gtdb']
species_assignment['assigned_species'] = species_assignment['species_gtdb'] + ' ' + species_assignment['assigned_species']
species_assignment['assigned_species'] = species_assignment['assigned_species'].str.strip()
for idx in df_genomes_meta[df_genomes_meta.genus_gtdb == "g__"].index:
    species_assignment.loc[idx, 'assigned_species'] = "N/A"
    species_assignment.loc[idx, 'species_gtdb'] = "N/A"
    species_assignment.loc[idx, 'genus_gtdb'] = "N/A"
df_species_assignment = species_assignment.rename(columns={"Mash_species" : "MASH_species"}).set_index("genome_id")

In [ ]:
df_final = df.set_index("strain_name").copy()
outfile = Path("../tables/Supplementary.xlsx")
outfile.parent.mkdir(exist_ok=True, parents=True)
df_final = df_final.merge(df_species_assignment, left_index=True, right_index=True)
df_final.to_excel(outfile)
#df_final